# NeRF Training on Google Colab (GitHub Version)

This notebook trains NeRF on the LEGO dataset using files from GitHub.

**Expected Runtime**: 4-6 hours on Colab GPU (T4)

**GitHub Repo**: https://github.com/mabuhann/nerf-pytorch

## Step 1: Check GPU

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

## Step 2: Clone GitHub Repository

In [ ]:
# Clone the GitHub repository
!git clone https://github.com/mabuhann/nerf-pytorch.git

print("\n✓ Repository cloned")

## Step 3: Navigate to Repository

In [ ]:
import os

# Change to repository directory
os.chdir('/content/nerf-pytorch')

print(f"Current directory: {os.getcwd()}")
print("\nFiles in directory:")
!ls -lh

## Step 4: Install Dependencies

In [ ]:
# Install required packages
!pip install imageio imageio-ffmpeg configargparse scikit-image tqdm matplotlib -q

print("✓ Dependencies installed")

## Step 5: Verify Dataset and Files

In [ ]:
import json

print("Checking required files...\n")

# Check Python files
required_files = [
    'run_nerf_with_metrics.py',
    'run_nerf_helpers.py',
    'load_blender.py'
]

for f in required_files:
    if os.path.exists(f):
        print(f"✓ {f}")
    else:
        print(f"✗ {f} NOT FOUND")

# Check config
if os.path.exists('configs/lego_config.txt'):
    print(f"✓ configs/lego_config.txt")
elif os.path.exists('lego_config.txt'):
    print(f"✓ lego_config.txt (will move to configs/)")
    !mkdir -p configs
    !mv lego_config.txt configs/
else:
    print(f"✗ lego_config.txt NOT FOUND")

# Check dataset
print("\nChecking dataset...\n")
base_path = 'data/nerf_synthetic/lego'

if os.path.exists(base_path):
    for split in ['train', 'val', 'test']:
        transform_file = f'{base_path}/transforms_{split}.json'
        if os.path.exists(transform_file):
            with open(transform_file, 'r') as f:
                data = json.load(f)
            n_frames = len(data['frames'])
            print(f"✓ {split}: {n_frames} images")
        else:
            print(f"✗ {transform_file} not found")
else:
    print(f"✗ Dataset not found at {base_path}")
    print("\nLooking for dataset in other locations...")
    !find . -name "transforms_train.json" -type f

## Step 6: Preview Configuration

In [ ]:
# Show the configuration
print("Configuration for training:\n")
if os.path.exists('configs/lego_config.txt'):
    !cat configs/lego_config.txt
else:
    print("Config file not found. Creating default config...")
    !mkdir -p configs
    
    config = """expname = lego_metrics
basedir = ./logs
datadir = ./data/nerf_synthetic/lego
dataset_type = blender

no_batching = True

use_viewdirs = True
white_bkgd = True
lrate_decay = 500

N_samples = 64
N_importance = 128
N_rand = 1024
chunk = 8192
netchunk = 131072

half_res = False

i_print = 100
i_weights = 10000
i_testset = 25000
i_video = 50000
"""
    
    with open('configs/lego_config.txt', 'w') as f:
        f.write(config)
    
    print("✓ Created default config")
    !cat configs/lego_config.txt

## Step 7: Start Training

**⚠️ IMPORTANT**: This will take 4-6 hours. Make sure:
1. You're using a GPU runtime (Runtime → Change runtime type → GPU)
2. Keep the tab open or use Colab Pro to avoid timeouts

The training will:
- Run for 200,000 iterations
- Save checkpoints every 10,000 iterations
- Evaluate on test set every 25,000 iterations
- Print progress every 100 iterations
- Save all metrics automatically

In [ ]:
# Start training
!python run_nerf_with_metrics.py --config configs/lego_config.txt

## Step 8: View Results

In [ ]:
import json
from IPython.display import Image, display

# Load and display summary metrics
print("="*60)
print("TRAINING SUMMARY")
print("="*60)

with open('logs/lego_metrics/summary_metrics.json', 'r') as f:
    summary = json.load(f)

for key, value in summary.items():
    if value is not None:
        print(f"{key}: {value}")

print("="*60)

# Display plots
print("\nTraining Metrics Plot:")
if os.path.exists('logs/lego_metrics/training_metrics.png'):
    display(Image('logs/lego_metrics/training_metrics.png'))
else:
    print("Plot not found")

## Step 9: Analyze Results (Optional)

In [ ]:
# Run detailed analysis if analyze_results.py exists
if os.path.exists('analyze_results.py'):
    !python analyze_results.py \
        --log_dir logs/lego_metrics \
        --save_plot logs/lego_metrics/detailed_analysis.png \
        --export_table logs/lego_metrics/results_table.md
    
    # Display detailed analysis
    print("\nDetailed Analysis:")
    if os.path.exists('logs/lego_metrics/detailed_analysis.png'):
        display(Image('logs/lego_metrics/detailed_analysis.png'))
else:
    print("analyze_results.py not found, skipping detailed analysis")

## Step 10: Download Results

In [ ]:
from google.colab import files

# Zip all results
!cd logs && zip -r lego_metrics_results.zip lego_metrics/

print("✓ Results zipped")
print("\nDownloading results...")

# Download the zip file
files.download('logs/lego_metrics_results.zip')

print("\n✓ Download started")
print("\nKey files in the zip:")
!zipinfo -1 logs/lego_metrics_results.zip | head -20

## Step 11: Quick Metrics Summary

In [ ]:
# Quick extraction of key metrics for your report
import json

with open('logs/lego_metrics/summary_metrics.json', 'r') as f:
    metrics = json.load(f)

print("="*60)
print("KEY METRICS FOR YOUR REPORT")
print("="*60)
print(f"\nTraining Time: {metrics.get('total_training_time_hours', 'N/A'):.2f} hours" if metrics.get('total_training_time_hours') else "Training Time: N/A")
print(f"Test PSNR: {metrics.get('best_test_psnr', 'N/A'):.2f} dB" if metrics.get('best_test_psnr') else "Test PSNR: N/A")
print(f"Test SSIM: {metrics.get('avg_test_ssim', 'N/A'):.4f}" if metrics.get('avg_test_ssim') else "Test SSIM: N/A")
print(f"Avg Step Time: {metrics.get('avg_step_time', 'N/A'):.4f} seconds" if metrics.get('avg_step_time') else "Avg Step Time: N/A")
print("\n" + "="*60)
print("\nCopy these values for comparison with 3DGS!")

## Troubleshooting

### Out of Memory Error
If you get OOM errors, reduce batch sizes. Create a new cell and run:
```python
# Edit config to reduce memory usage
config = open('configs/lego_config.txt').read()
config = config.replace('N_rand = 1024', 'N_rand = 512')
config = config.replace('chunk = 8192', 'chunk = 4096')
with open('configs/lego_config.txt', 'w') as f:
    f.write(config)
print("✓ Config updated with lower memory settings")
```

### Session Timeout
- Use Colab Pro for longer sessions
- Keep browser tab active
- Consider reducing iterations for testing

### Dataset Not Found
Make sure your GitHub repo has the data in the correct location:
- `data/nerf_synthetic/lego/transforms_train.json`
- `data/nerf_synthetic/lego/train/` folder with images